In [31]:
import pandas as pd
import numpy as np
import re

In [32]:
data = pd.read_csv('../data/combined_data_v3.csv')
data.shape

(389504, 5)

In [33]:
guardian_length = data[data.source == 'guardian'].shape[0]

In [34]:
Huff = data[data.source == 'Huffington Post']
ABC = data[data.source == 'ABC Australia']

In [35]:
other_sources = data[(data.source != 'Huffington Post') & (data.source != 'ABC Australia')]

In [36]:
Huff = Huff.sample(n=guardian_length)
ABC = ABC.sample(n=guardian_length)

In [37]:
data_sources = [Huff, ABC, other_sources]
data = pd.concat(data_sources)
data.reset_index(drop=True, inplace = True)
data.shape

(181717, 5)

In [38]:
data.sample(n = 10)

,source,headline,year,date,is_sarcastic
143142,atlantic,James Comey's 'Shock and Awe' Testimony,2017,2017-06-08,0
10001,Huffington Post,Kelly Ripa Will Be Back On Tuesday,2016,2016-04-23,0
17864,Huffington Post,Millions Of Kids Might Lose Health Care Becaus...,2017,2017-12-20,0
25220,Huffington Post,black women are rising – when will our pay?,2019,2019-01-01,0
65464,ABC Australia,cryptocurrency for selling your dna online,2018,2018-01-23,0
57366,ABC Australia,nick kyrgios escapes with suspended ban for ci...,2019,2019-09-27,0
46744,ABC Australia,tafe sa failings laid bare as courses given al...,2018,2018-04-04,0
39127,ABC Australia,cloud over perth royal show wont force date ch...,2017,2017-10-10,0
110331,guardian,Families torn apart: the anatomy of Trump's im...,2018,2018-06-23,0
22475,Huffington Post,sia's christmas album title contains an awkwar...,2019,2019-01-01,0


### Preprocessing Steps
1. Standardize common abbreviations* (WIP)
    1. u.s. --> usa
1. Lowercase
1. Expand Contractions
1. Optional:
    1. Remove Source Specific Language
    1. Remove Profanity
1. Remove Special Characters


In [39]:
def replace_words(text, replace_dict):
    tokens = []
    for w in text.split():
        word = w
        for t in replace_dict.keys():
            if w == t:
                word = replace_dict[t]
        tokens.append(word)
    
    return " ".join(tokens)

In [40]:
data['clean'] = data['headline']

## Standardize Common Abbreviations
translate_dict = {
    "US": "USA",
    "U.S.": "USA",
    "u.s.": "USA",
    "u.s": "USA",
}



data['clean'] = data['clean'].apply(lambda x: replace_words(x, translate_dict))

In [41]:
# Lowercase
data['clean'] = data['clean'].apply(lambda x: x.lower())

In [42]:
# Expand Contractions
contractions_dict = { "ain't": "are not","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who's": "who is",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}



data['clean'] = data['clean'].apply(lambda x: replace_words(x, contractions_dict))


In [43]:
# Example should read: who is afraid
data['clean'][77627]

'french president francois hollande says he will not seek re-election'

In [44]:
## Remove Special Characters
data['clean'] = data['clean'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', '', x))

In [45]:
data['clean']

0         saudi arabias public cinema ban lifts with a s...
1         maryland police charge 3 church leaders with p...
2         manatees no longer facing extinction but there...
3         usa lashed as main threat to environment at un...
4         olympian proudly says black girls rock as she ...
                                ...                        
181712    overworked gamer wishes he could just stay hom...
181713    how king henry viii became the worlds first ga...
181714    ring fit user hates how crowded adventure gets...
181715    gamer postpones new years resolution until q3 ...
181716    netflix announces its losing the audio portion...
Name: clean, Length: 181717, dtype: object

In [46]:
data[['source', 'year', 'clean', 'is_sarcastic']].to_csv('../data/combined_clean.csv', index = False, sep = "|")

In [49]:
df = data[['source', 'year', 'clean', 'is_sarcastic']]

In [50]:
df.to

,source,year,clean,is_sarcastic
0,Huffington Post,2018,saudi arabias public cinema ban lifts with a s...,0
1,Huffington Post,2018,maryland police charge 3 church leaders with p...,0
2,Huffington Post,2016,manatees no longer facing extinction but there...,0
3,Huffington Post,2017,usa lashed as main threat to environment at un...,0
4,Huffington Post,2016,olympian proudly says black girls rock as she ...,0
...,...,...,...,...
181712,Hard Times,2020,overworked gamer wishes he could just stay hom...,1
181713,Hard Times,2020,how king henry viii became the worlds first ga...,1
181714,Hard Times,2020,ring fit user hates how crowded adventure gets...,1
181715,Hard Times,2020,gamer postpones new years resolution until q3 ...,1
